# Import Libraries

In [ ]:
!pip install pandas numpy scikit-learn joblib pickle5

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pickle5
  Running setup.py clean for pickle5
Failed to build pickle5
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pickle5)


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

#Load Dataset

In [ ]:
df=pd.read_csv("/content/swiggy.csv")
df

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json
...,...,...,...,...,...,...,...,...,...,...,...
148536,553122,The Food Delight,Yavatmal,--,Too Few Ratings,₹ 200,"Fast Food,Snacks",21522053000452,https://www.swiggy.com/restaurants/the-food-de...,"The Food Delight, 94MC+X35, New Singhania Naga...",Menu/553122.json
148537,562647,MAITRI FOODS & BEVERAGES,Yavatmal,--,Too Few Ratings,₹ 300,Pizzas,license,https://www.swiggy.com/restaurants/maitri-food...,"MAITRI FOODS & BEVERAGES, POLIC MITRYA SOCIETY...",Menu/562647.json
148538,559435,Cafe Bella Ciao,Yavatmal,--,Too Few Ratings,₹ 300,"Fast Food,Snacks",21522251000378,https://www.swiggy.com/restaurants/cafe-bella-...,"Cafe Bella Ciao, SHOP NO 2 NEMANI MARKET SBI S...",Menu/559435.json
148539,418989,GRILL ZILLA,Yavatmal,--,Too Few Ratings,₹ 250,Continental,21521251000241,https://www.swiggy.com/restaurants/grill-zilla...,"GRILL ZILLA, SHO NO 2/6, POSTEL GROUND CHOWPAT...",Menu/418989.json


In [ ]:
df.shape

(148541, 11)

In [ ]:
df.head()

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


#  Data Cleaning

In [ ]:
df = df.drop_duplicates()

In [ ]:
text_cols = ['name', 'city', 'cuisine', 'address', 'menu', 'link', 'lic_no']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

## Handle missing values


In [ ]:
# Fill missing categorical columns

for col in ['name', 'city', 'cuisine']:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

In [ ]:
# Convert numerical columns

for col in ['rating', 'rating_count', 'cost']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
# Fill numeric missing values

df['rating'] = df['rating'].fillna(df['rating'].median())
df['rating_count'] = df['rating_count'].fillna(df['rating_count'].median() if df['rating_count'].notnull().sum() > 0 else 0)
df['cost'] = df['cost'].fillna(df['cost'].median() if df['cost'].notnull().sum() > 0 else 300)

In [ ]:
if 'lic_no' in df.columns:
    df['lic_no'] = df['lic_no'].replace('nan', np.nan).fillna("Not Available")

In [ ]:
df.to_csv("cleaned_data.csv", index=False)

In [ ]:
print(df.isnull().sum())

id              0
name            0
city            0
rating          0
rating_count    0
cost            0
cuisine         0
lic_no          0
link            0
address         0
menu            0
dtype: int64


# Data Preprocessing (One-Hot Encoding)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pickle

In [ ]:
categorical_cols = ['city', 'cuisine']
numeric_cols = ['rating', 'rating_count', 'cost']

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

In [ ]:
encoded_cat = encoder.fit_transform(df[categorical_cols])

In [ ]:
print("Encoded categorical shape:", encoded_cat.shape)

Encoded categorical shape: (148541, 2953)


In [ ]:
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

In [ ]:
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoded_feature_names)

In [ ]:
encoded_df = pd.concat(
    [df[numeric_cols].reset_index(drop=True), encoded_cat_df.reset_index(drop=True)],
    axis=1
)

In [ ]:
encoded_df.to_csv("encoded_data.csv", index=False)

In [ ]:
with open("encoder.pkl", "wb") as f:
    pickle.dump({
        "encoder": encoder,
        "categorical_cols": categorical_cols,
        "numeric_cols": numeric_cols,
        "feature_names": encoded_feature_names.tolist()
    }, f)

In [ ]:
print("Encoded data shape:", encoded_df.shape)

Encoded data shape: (148541, 2956)


#  Cosine Similarity Based Recommendation System

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

In [ ]:
encoded_df = pd.read_csv("/content/encoded_data.csv")

In [ ]:
clean_df = pd.read_csv("/content/cleaned_data.csv")

In [ ]:
with open("encoder.pkl", "rb") as f:
    meta = pickle.load(f)

In [ ]:
encoder = meta["encoder"]
categorical_cols = meta["categorical_cols"]
numeric_cols = meta["numeric_cols"]

# Build User Vector &  Generate Top-N Recommendations

In [ ]:
def build_user_vector(user_input):
    # Extract numeric inputs
    num_values = [float(user_input.get(col, 0)) for col in numeric_cols]

    # Extract categorical inputs
    cat_values = [user_input.get(col, "Unknown") for col in categorical_cols]
    cat_encoded = encoder.transform([cat_values])[0]

    # Combine numeric + categorical
    user_vector = np.concatenate([num_values, cat_encoded])
    return user_vector


In [ ]:
def recommend_restaurants(user_input, top_n=5):
    user_vector = build_user_vector(user_input)

    # Compute cosine similarity
    similarities = cosine_similarity([user_vector], encoded_df)[0]

    # Get top N similar indices
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Return recommendations from the clean dataset
    recommendations = clean_df.iloc[top_indices][['name', 'city', 'cuisine', 'rating', 'cost', 'address', 'link']]
    recommendations['similarity_score'] = similarities[top_indices]

    return recommendations

In [ ]:
sample_user = {
    "city": "Bangalore",
    "cuisine": "North Indian",
    "rating": 4.0,
    "rating_count": 50,
    "cost": 300
}

In [ ]:
recommendations = recommend_restaurants(sample_user, top_n=5)
print("✅ Top 5 Recommended Restaurants for You:")
recommendations

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


✅ Top 5 Recommended Restaurants for You:


,name,city,cuisine,rating,cost,address,link,similarity_score
22577,FOODIES CORNER,"Kadubeesanahalli,Bangalore",North Indian,4.0,300.0,"FOODIES CORNER, NO 55/1 SHIRIDI SAI LAYOUT NEA...",https://www.swiggy.com/restaurants/foodies-cor...,0.986391
70282,Lazeez Moments,Hansi,North Indian,4.0,300.0,"Lazeez Moments, Delhi Chungi, Hansi",https://www.swiggy.com/restaurants/lazeez-mome...,0.986391
127824,Hotel Hariom,"Narhe,Pune",North Indian,4.0,300.0,"Hotel Hariom, Front of Samadhan Bhel, Near Avd...",https://www.swiggy.com/restaurants/hotel-hario...,0.986391
32486,Arabic Restro Bar Lounge,"North Mohali,Chandigarh",North Indian,4.0,300.0,"Arabic Restro Bar Lounge, SCO 670, Sector 70, ...",https://www.swiggy.com/restaurants/arabic-rest...,0.986391
49499,Chaman Dhaba,"Laxmi Nagar,Delhi",North Indian,4.0,300.0,"Chaman Dhaba, 450/1 Main road jheel khurenja, ...",https://www.swiggy.com/restaurants/chaman-dhab...,0.986391
